In [2]:
# What version of Python do you have?
import sys
import os
import tensorflow
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
Pandas 1.3.3
Scikit-Learn 0.24.2
GPU is available


In [3]:
from tensorflow.keras.applications import ResNet152, InceptionV3

In [4]:
inceptionV3 = InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape = (299, 299, 3)
)

for layer in inceptionV3.layers:
    layer.trainable = True

In [5]:
inceptionV3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
resnet152 = ResNet152(
    weights='imagenet',
    include_top=False,
    input_shape=((224, 224, 3))
)

for layer in resnet152.layers:
    layer.trainable = True

reshaping = layers.Reshape((8, 8, 1568))(resnet152.output)
re_config_resnet152 = tensorflow.keras.Model(inputs=resnet152.input, outputs=reshaping)

In [7]:
a = np.zeros(shape=(7, 7, 2048))
a = a.reshape(8, 8, 1568)
a.shape

(8, 8, 1568)

In [8]:
re_config_resnet152.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:

# sort losses of batch descending order
# the largest k? is averaged as the final loss
# return loss
# parameters needed: SUM, label(i), loss of batches


def ohem_loss(ytrue, ypred):
    result = K.categorical_crossentropy(ytrue, ypred, axis=-1)
    alpha = K.variable(0.1, dtype=tf.float32)
    index = K.greater(result, alpha)
    cast = tf.cast(index, tf.float32)
    return result*cast

In [10]:
from tensorflow.keras import Model
from tensorflow.keras import backend as K


concat_feature_layer = layers.concatenate([inceptionV3.output, re_config_resnet152.output])
fully_connected_dense_big = layers.Dense(1024, activation='relu')(concat_feature_layer)
dropout_one = layers.Dropout(0.5)(fully_connected_dense_big)
flatten_layer = layers.Flatten()(dropout_one)
fully_connected_dense_small = layers.Dense(512, activation='relu')(flatten_layer)
dropout_two = layers.Dropout(0.5)(fully_connected_dense_small)
fully_connected_dense_class = layers.Dense(4, activation='softmax')(dropout_two)

model = Model(
    inputs=[inceptionV3.input, re_config_resnet152.input],
    outputs=fully_connected_dense_class
)

model.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
    
)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [11]:
modelplot = tensorflow.keras.utils.plot_model(model)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [12]:
print(os.getcwd())
path_to_windows = "./windows"

c:\Users\thoma\Desktop\speciale\DeepLearning_Master


In [13]:
resnet152_input_shape = (224, 224)
inceptionV3_input_shape = (299, 299)

In [14]:
labels = [item for item in os.listdir(path_to_windows)]
labels = np.array(labels)
labels = labels[0:]
print(labels.shape)
labels

(4,)


array(['Interictal', 'Preictal_One', 'Preictal_Two', 'Seizure'],
      dtype='<U12')

In [15]:
batch_size = 8

In [16]:
generator = ImageDataGenerator(
rescale = 1./255, 
validation_split=0.2
)

In [17]:
def generator_two_img(X1, y, batch_size, subset):
    genX1 = generator.flow_from_directory(X1, classes=y, batch_size=batch_size, subset=subset, target_size=inceptionV3_input_shape)
    genX2 = generator.flow_from_directory(X1, classes=y, batch_size=batch_size, subset=subset, target_size=resnet152_input_shape)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield [X1i[0], X2i[0]], X1i[1]

In [18]:
train_steps = generator.flow_from_directory(path_to_windows, classes=list(labels), batch_size=batch_size, subset="training").samples // batch_size

validation_steps = generator.flow_from_directory(path_to_windows, classes=list(labels), batch_size=batch_size, subset="validation").samples // batch_size

Found 594459 images belonging to 4 classes.
Found 148612 images belonging to 4 classes.


In [19]:
class CustomSaver(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch == 2:  # or save after some epoch, each k-th epoch etc.
            self.model.save("model_multi_channel.hd5".format(epoch))

saver = CustomSaver()

In [20]:
history = model.fit(
    x=generator_two_img(X1=path_to_windows, y=list(labels), batch_size=batch_size, subset="training"),
    epochs=3,
    batch_size=batch_size,
    steps_per_epoch = train_steps,
    validation_steps=validation_steps,
    validation_data=generator_two_img(X1=path_to_windows, y=list(labels), batch_size=batch_size, subset="validation"),
    callbacks=[saver]
)

Found 594459 images belonging to 4 classes.
Found 594459 images belonging to 4 classes.
Epoch 1/3
74307/74307 [==============================] - ETA: 0s - loss: 0.3059 - accuracy: 0.9366Found 148612 images belonging to 4 classes.
Found 148612 images belonging to 4 classes.
74307/74307 [==============================] - 36537s 491ms/step - loss: 0.3059 - accuracy: 0.9366 - val_loss: 0.2881 - val_accuracy: 0.9369
Epoch 2/3
74307/74307 [==============================] - 35714s 481ms/step - loss: 0.2883 - accuracy: 0.9369 - val_loss: 0.2880 - val_accuracy: 0.9369
Epoch 3/3
74307/74307 [==============================] - 35669s 480ms/step - loss: 0.2881 - accuracy: 0.9369 - val_loss: 0.2880 - val_accuracy: 0.9369


C:\Users\thoma\anaconda3\envs\py\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: model_multi_channel.hd5\assets


In [23]:
model.save("fneur_model")

C:\Users\thoma\anaconda3\envs\py\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: fneur_model\assets
